# Создание моделей машинного обучения в Scikit-Learn

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/ITMO
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler
df = pd.read_csv('train.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/MyDrive/ITMO'
/content/gdrive/MyDrive/ITMO


In [ ]:
# Проверяем дисбаланс между выжившими и невыживших пассажирами
S_counts = df['Survived'].value_counts()
S_percent = df['Survived'].value_counts(normalize=True) * 100

print("Количество выживших и невыживших:")
print(S_counts)

print("\nПроцентное соотношение:")
print(round(S_percent, 0))

Количество выживших и невыживших:
Survived
0    549
1    342
Name: count, dtype: int64

Процентное соотношение:
Survived
0    62.0
1    38.0
Name: proportion, dtype: float64


In [ ]:
## Как мы видим, поле 'Survived' в датасете train.csv (данне ые о выживаемости на Титанике) является дисбалансированным, а лучше всего с дисьбалансом справляется метрика ROC-AUC так как она измеряет ранжирование предсказаний, а не абсолютные значения.
## В отличие от метрик Precision, Recall или F1-score, которые требуют выбора конкретного порога для принятия решения, ROC-AUC использует всю кривую ROC, что позволяет оценить качество модели в целом без привязки к конкретному порогу.
## Таким образом по моему мнению наиболее релевантной метрикой для оценки выживаемости будет ROC-AUC

In [ ]:
## Выделяем целевую переменную и релевантные признаки
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = df[features]
y = df['Survived']

## Заполняем пропуски:
## - в поле 'Age' средним значением
mean_age = X['Age'].median()
X.loc[X['Age'].isnull(), 'Age'] = mean_age

## - в поле 'Embarked' наиболее часто встречающимся значением
mode_embarked = X['Embarked'].mode()[0]
X.loc[X['Embarked'].isnull(), 'Embarked'] = mode_embarked

## Обрабатываем категориальные признаки
X = pd.get_dummies(X, columns=['Sex', 'Embarked'], drop_first=True)

## Выполняем масштабирование признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## Выполняем разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2025, stratify=y)

In [ ]:
## Выполняем измерение качества константного предсказания (Наиболее частый класс в обучающей выборке)
most_frequent_class = y_train.mode()[0]
y_pred_const_majority = np.full_like(y_test, fill_value=most_frequent_class)
accuracy_const_majority = accuracy_score(y_test, y_pred_const_majority)
print(f"Качество константного предсказания (наиболее частый класс): {accuracy_const_majority:.3f}")

Качество константного предсказания (наиболее частый класс): 0.615


In [ ]:
## Выполняем обучение модели LogisticRegression с учетом предобработки:
model = LogisticRegression(random_state=2025, max_iter=1000)
model.fit(X_train, y_train)

# Выполянем оценку качества на тестовой выборке:
y_pred = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred)
print(f"Оценка модели по метрике ROC-AUC: {roc_auc:.3f}")

Оценка модели по метрике ROC-AUC: 0.833
